In [1]:
import pandas as pd

In [2]:
# ファイルの場所

input_data = 'input_files/JO2020.xlsx'
output_data = 'output_files/JTS_score_data.xlsx'

In [3]:
# データからの検索をするためのリスト

teams = [
    '金沢学院大学クラブ',
    'アベノジュニアトランポリンクラブ',
    '厚木FUSiONスポーツクラブ',
    '日本体育大学トランポリンクラブ',
    '星稜クラブ',
    'キタイスポーツクラブ'
]

names = [
    'ひかる',
    '裕美',
    '愛菜',
    '沙季',
    'セリナ',
    'ここね',
    '守杏',
    '晴茄',
    '希湖',
    '美咲希'
]

stages = [
    'Qualification',
    'Final'
]

In [4]:
# 各大会のスコアデータの読み込み時に必要なデータのみ抽出

df = pd.read_excel(input_data)
df = df[[
    'Competition',
    'Stage',
    'Surname',
    'Given Name',
    'Representing',
    'Mark',
    'Judge',
    '∑'
]]

FileNotFoundError: [Errno 2] No such file or directory: 'scoredata.xlsx'

In [192]:
# 上記のリストを利用し、クラブ名、名前、ルーティン名で検索して一人を抜き出して df1 に格納
# その後、各ジャッジの点数が列として格納されているため、一旦ジャッジ名と点数のみを取り出して転置し df2 に格納
# さらに、df1 と df2 のインデックスを一旦クリアし結合処理

def main(team, name, stage):
    df1 = df[df['Representing'] == team]
    df1 = df1[df1['Given Name'] == name]
    df1 = df1[df1['Stage'] == stage]
    df1 = df1.iloc[:1, [0, 1, 2, 3, 4, 5]]

    df2= df[df['Representing'] == team]
    df2 = df2[df2['Given Name'] == name]
    df2 = df2[df2['Stage'] == stage]
    df2 = df2.loc[:, ['Judge','∑']]
    df2 = df2.set_index('Judge')
    df2 = df2[0:10].T
    
    df1.reset_index(drop=True, inplace=True)
    df2.reset_index(drop=True, inplace=True)

    df3 = pd.concat([df1, df2], axis=1)
    return df3

In [193]:
# result_df という空のリストを用意
# for文にて、各選手の情報を検索し、main() に渡す
# 取り出された各選手のデータを result_df に格納
# result_df に格納された各選手のデータを結合処理

result_df = []

for team in teams:
    for name in names:
        for stage in stages:
            result_df.append(main(team, name, stage))

result_df = pd.concat(result_df)

In [194]:
# 結合時にできた欠損値を含む行をすべて削除

result_df = result_df.dropna(how='all')

In [195]:
# lambda を使って、各列のスコアの桁数を調整

result_df['Mark'] = result_df.apply(lambda x: x['Mark']/1000, axis=1)
result_df['E1'] = result_df.apply(lambda x: x['E1']/10, axis=1)
result_df['E2'] = result_df.apply(lambda x: x['E2']/10, axis=1)
result_df['E3'] = result_df.apply(lambda x: x['E3']/10, axis=1)
result_df['E4'] = result_df.apply(lambda x: x['E4']/10, axis=1)
result_df['E5'] = result_df.apply(lambda x: x['E5']/10, axis=1)
result_df['E8'] = result_df.apply(lambda x: x['E8']/10, axis=1)
result_df['E∑'] = result_df.apply(lambda x: x['E∑']/10, axis=1)
result_df['T'] = result_df.apply(lambda x: x['T']/1000, axis=1)
result_df['D'] = result_df.apply(lambda x: x['D']/10, axis=1)
result_df['H'] = result_df.apply(lambda x: x['H']/100, axis=1)

In [196]:
# Excel ファイルに保存

result_df.to_excel(output_data, index=False)

In [197]:
result_df

,Competition,Stage,Surname,Given Name,Representing,Mark,E1,E2,E3,E4,E5,E8,E∑,T,D,H
0,ジャパンオープン/全日本社会人 女子 (B台),Qualification,森,ひかる,金沢学院大学クラブ,50.025,9.1,9.4,9.3,9.3,9.1,9.2,18.6,16.725,5.0,9.7
0,ジャパンオープン/全日本社会人 女子 (B台),Final,森,ひかる,金沢学院大学クラブ,54.235,8.4,7.9,8.3,8.1,7.9,7.9,16.1,15.935,12.8,9.4
0,ジャパンオープン/全日本社会人 女子 (B台),Qualification,高木,裕美,金沢学院大学クラブ,47.785,9.1,9.0,9.0,8.8,8.4,9.0,17.9,15.785,4.6,9.5
0,ジャパンオープン/全日本社会人 女子 (B台),Qualification,田中,沙季,金沢学院大学クラブ,20.130,3.1,3.4,3.4,3.3,3.3,3.4,6.7,6.530,3.1,3.8
0,13-14才 女子 (B台),Qualification,田中,希湖,アベノジュニアトランポリンクラブ,52.320,7.9,8.3,8.6,8.4,8.3,8.4,16.8,15.020,11.1,9.4
0,13-14才 女子 (B台),Final,田中,希湖,アベノジュニアトランポリンクラブ,30.990,4.7,5.4,5.1,4.9,5.0,5.0,10.0,9.190,6.8,5.0
0,13-14才 女子 (B台),Qualification,石田,美咲希,厚木FUSiONスポーツクラブ,35.870,5.8,5.7,5.7,5.7,5.7,5.6,11.4,10.470,7.5,6.5
0,15-16才 女子 (A台),Qualification,小野,晴茄,日本体育大学トランポリンクラブ,49.615,8.1,8.0,8.2,8.5,8.1,8.1,16.2,15.615,8.6,9.2
0,15-16才 女子 (A台),Final,小野,晴茄,日本体育大学トランポリンクラブ,50.585,8.3,8.0,8.4,8.2,8.4,8.5,16.5,15.585,8.8,9.7
